In [72]:
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Embedding, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

import matplotlib.pyplot as plt


In [73]:
docs = ['먼저 덱스트의 각 단어를 나누어 토큰화합니다.',
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
        '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']

In [74]:
token = Tokenizer()
token.fit_on_texts(docs)

print('텍스트 단어는: ', token.word_counts)

텍스트 단어는:  OrderedDict([('먼저', 1), ('덱스트의', 1), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('텍스트의', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [75]:
print(token.texts_to_sequences(docs))




[[2, 3, 4, 5, 6, 7], [8, 9, 10, 1, 11], [12, 13, 1, 14, 15, 16]]


In [76]:
text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'
token1 = Tokenizer()
token1.fit_on_texts([text])
x = token1.texts_to_sequences([text])
print(token1.texts_to_sequences([text]))

[[1, 2, 3, 4, 5, 6]]


In [77]:
to_categorical(x, len(token.word_index)+1)

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]]], dtype=float32)

In [78]:
# 영화 리뷰가 긍정인지 부정인지 예측

review = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]
classes = np.array([1,1,1,1,1,0,0,0,0,0])

# 토큰화
token = Tokenizer()
token.fit_on_texts(review)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [79]:
# 토큰한 결과를 가지고 인덱스로 문장을 변환

x = token.texts_to_sequences(review)
print(x)

[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [80]:
# 각 ㅜㄴ장의 단어의 갯수가 다음 -> 문장의 단어의 개수를 동일하게 적용
padd_x = pad_sequences(x, 4)    # x를 각 문장의 단어 수가 4개로 통일
print(padd_x)

[[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [81]:
model = Sequential()
model.add(Embedding(len(token.word_index)+1, 8, input_length=4))    # 총단어수, 임베딩사이즈, 입력 단어수
model.add(Flatten())    # 다음 레이어를 위해 1차원 배열로 변환
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 4, 8)              168       
                                                                 
 flatten_4 (Flatten)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [82]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padd_x, classes, epochs=20)

print('test 정확도:', model.evaluate(padd_x, classes)[1])

Epoch 1/20
1/1 [==============================] - 1s 722ms/step - loss: 0.7039 - accuracy: 0.3000
Epoch 2/20
1/1 [==============================] - 0s 19ms/step - loss: 0.7018 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 26ms/step - loss: 0.6997 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 22ms/step - loss: 0.6977 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 21ms/step - loss: 0.6956 - accuracy: 0.6000
Epoch 6/20
1/1 [==============================] - 0s 20ms/step - loss: 0.6935 - accuracy: 0.6000
Epoch 7/20
1/1 [==============================] - 0s 25ms/step - loss: 0.6915 - accuracy: 0.7000
Epoch 8/20
1/1 [==============================] - 0s 29ms/step - loss: 0.6895 - accuracy: 0.7000
Epoch 9/20
1/1 [==============================] - 0s 29ms/step - loss: 0.6874 - accuracy: 0.7000
Epoch 10/20
1/1 [==============================] - 0s 17ms/step - loss: 0.6854 - accuracy: 0.7000
Epoch 11/20
1/1 [===========